# 🍜 頑固ラーメン屋のAIを作ろう (Build a Stubborn Ramen AI)

---
```text
 FFFFF   A   QQQQ      BBBB   OOO  TTTTT
 F      A A  Q  Q      B  B  O   O   T
 FFF   AAAAA Q  Q      BBBB  O   O   T
 F     A   A Q  Q      B  B  O   O   T
 F     A   A  QQ Q     BBBB   OOO    T
```
#  「初めての言語モデル構築」ラボへようこそ！
---
このワークショップでは、Pythonを使って **頑固なラーメン屋の店長** のようなAIをゼロから作ります。
最初は何も知らないAIですが、学習（トレーニング）させることで、ラーメン屋らしい会話ができるようになります。

# 📖 今日の辞書 (Concept Dictionary)
難しい言葉が出てきたら、ここを見てください。

| 言葉 (Term) | 意味 (Meaning) | 例え (Analogy) |
| :--- | :--- | :--- |
| **Tokens (トークン)** | AIが読む文字の単位 | 「あ」「い」「う」など、文字ひとつひとつ |
| **Batch Size (バッチサイズ)** | 一度に勉強する量 | 英単語カードを「5枚ずつ」めくるか、「30枚ずつ」めくるか |
| **Loss (ロス・損失)** | AIの間違いの数 | テストの「バツ」の数。**0に近いほど賢い！** |
| **Optimizer (オプティマイザ)** | AIを修正する先生 | 「そこ間違ってるよ、こう直しなさい」と指導する役割 |
| **Iteration (イテレーション)** | 学習の回数 | ドリルを繰り返す回数 |

---


# ==============================================================================
# ✅ パート０：ライブラリの読み込み
# ==============================================================================
 ニューラルネットワークの構築にはPyTorchを、UIにはipywidgetsを使用します。

In [1]:
import warnings
# Filter out the specific warning about pynvml
warnings.filterwarnings("ignore", category=FutureWarning)

import torch
import torch.nn as nn
from torch.nn import functional as F

# ==============================================================================
# ✅ パート1：教科書（データ）の準備
# ==============================================================================

AIに読ませる「頑固ラーメン屋の会話集」です。
AIはこのテキストのパターンを真似するように学習します。

In [2]:
text = """
客：いらっしゃい。
店：へい、いらっしゃい。食券買ってな。
客：おすすめは何ですか？
店：うちは塩ラーメンしか置いてないよ。メニューをよく見なさい。
客：すいません、お水ください。
店：水はセルフサービスだよ。あそこの給水機を使ってくれ。
客：大盛りはできますか？
店：うちは大盛りやってないんだ。味のバランスが崩れるからね。
客：麺の硬さは選べますか？
店：うちは「普通」が一番うまいんだ。黙って座って待ってな。
客：ごちそうさまでした。
店：おう、まいど。丼はカウンターに上げてってな。
客：トイレはどこですか？
店：店の外を出て右だよ。
客：替え玉お願いします。
店：だから、メニュー見てくれよ。替え玉もやってないんだ。
""" * 100  # データ量を増やすために100回繰り返します

print(f"データの文字数: {len(text)} 文字")

データの文字数: 31500 文字


In [3]:
### ------------------------------------------------------------------------------
### ここで「文字」を「数字」に変換します (AIは計算しかできないため)
### ------------------------------------------------------------------------------

### データの中にどんな文字があるかリストアップします
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(f"使われている文字の種類: {len(chars)} 種類")
print(f"文字リスト: {''.join(chars)}")

### 文字と数字の対応表を作ります (Encoding / Decoding)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]          # 文字 -> 数字
decode = lambda l: ''.join([itos[i] for i in l]) # 数字 -> 文字

print("\n--- 変換テスト ---")
print(f"元の言葉: おすすめ")
print(f"AIの視点: {encode('おすすめ')}")

使われている文字の種類: 104 種類
文字リスト: 
、。「」あいうえおかがきくげこごさしすせそただちってでどなにねのはへべまめもゃやよらりるれをんイウカサスセタトニバビフメュラルレンー一上丼何使出券右味塩外大客崩店座待普替機水玉番盛硬給置見買通選願食麺黙：？

--- 変換テスト ---
元の言葉: おすすめ
AIの視点: [9, 19, 19, 37]


# ==============================================================================
# ✅ パート2：AIの「脳」を構築する 
# ※ このセルは変更しなくてOKです。実行(Shift+Enter)だけしてください。
# ==============================================================================

In [4]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # 各トークン（文字）の確率を読むためのテーブルを作成
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) 
        
        if targets is None:
            loss = None
        else:
            # 予測(logits)と正解(targets)を比べて、誤差(loss)を計算する
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # 現在の文字から、次の文字を予測して生成するループ
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# データをバッチ（小分け）にする関数
def get_batch(split):
    data = torch.tensor(encode(text), dtype=torch.long)
    ix = torch.randint(len(data) - 8, (32,))
    x = torch.stack([data[i:i+8] for i in ix])
    y = torch.stack([data[i+1:i+8+1] for i in ix])
    return x, y

print("AIの脳の「設計図」が完成しました！")

AIの脳の「設計図」が完成しました！


# ==============================================================================
# 🔧 パート3：AIのトレーニング (★ ここが今日の主役です！) 
# ==============================================================================

ここから皆さんの出番です。
以下のコードには **空欄（______）** があります。
ヒントを読みながら、正しいコードや数字を入れてAIを学習させてください。

In [5]:
# ---------------------------------------------------------
# 🧐 先生用の解説コーナー (Instructor Demo)
# AIが何を「正解」としているか確認しましょう
# ---------------------------------------------------------

x_demo, y_demo = get_batch('train')

print("例：ラーメン屋のデータの「入力」と「正解」の関係\n")

# 最初のバッチの、最初の4文字だけを見てみます
for i in range(4):
    input_text = decode([x_demo[0, i].item()])
    target_text = decode([y_demo[0, i].item()])
    
    print(f"ステップ {i+1}:")
    print(f"  AIが見ている文字 (Input):  '{input_text}'")
    print(f"  AIが当てるべき文字 (Target): '{target_text}'")
    print("-------------------------")

例：ラーメン屋のデータの「入力」と「正解」の関係

ステップ 1:
  AIが見ている文字 (Input):  'だ'
  AIが当てるべき文字 (Target): 'よ'
-------------------------
ステップ 2:
  AIが見ている文字 (Input):  'よ'
  AIが当てるべき文字 (Target): '。'
-------------------------
ステップ 3:
  AIが見ている文字 (Input):  '。'
  AIが当てるべき文字 (Target): '
'
-------------------------
ステップ 4:
  AIが見ている文字 (Input):  '
'
  AIが当てるべき文字 (Target): '客'
-------------------------


In [6]:
# モデルを初期化します
model = BigramLanguageModel(vocab_size)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

# -----------------------------------------------------------------
# 実験コーナー：パラメータを設定しよう (Fill in the blanks!)
# -----------------------------------------------------------------

# 学習回数 (AIがドリルを解く回数)
# ヒント: 100回だと全然足りませんが、3000回やると少し賢くなります。
# 推奨: 最初は 100 で試して、後で 3000 に書き換えてみましょう。
max_iters = 3000 


print(f"{max_iters}回の学習を開始します...")

for iter in range(max_iters):
    # データを取得
    xb, yb = get_batch('train')

    # -------------------------------------------------------------
    # クイズ：学習ループを完成させよう (ヒントを見て選んでください)
    # -------------------------------------------------------------
    
    # 1. 誤差(Loss)を計算する
    logits, loss = model(xb, yb)
    
    # 2. 前回の計算の「ゴミ」を掃除する呪文
    # ヒント: [ 選択肢: optimizer.zero_grad(set_to_none=True)  または  print("hello") ]
    # -------------------------------------------------------------
    optimizer.zero_grad(set_to_none=True) 

    # 3. バックプロパゲーション (間違いの原因を探る)
    loss.backward()
    
    # 4. パラメータを実際に更新する（先生が修正する）呪文
    # ヒント: [ 選択肢: optimizer.step()  または  optimizer.stop() ]
    # -------------------------------------------------------------
    optimizer.step() 

    if iter % 500 == 0:
        print(f"回数: {iter}, 誤差(Loss): {loss.item():.4f}")

print("学習完了！")

3000回の学習を開始します...
回数: 0, 誤差(Loss): 5.2975
回数: 500, 誤差(Loss): 4.5356
回数: 1000, 誤差(Loss): 3.7590
回数: 1500, 誤差(Loss): 3.2295
回数: 2000, 誤差(Loss): 2.5521
回数: 2500, 誤差(Loss): 2.0761
学習完了！


# ==============================================================================
# ✅ パート4：AI店長と話してみよう
# ==============================================================================

In [7]:
print("--- AI店長の生成テキスト ---")

# 何もない状態(0)からスタートして、300文字生成させます
context = torch.zeros((1, 1), dtype=torch.long)
generated_output = model.generate(context, max_new_tokens=300)[0].tolist()

# 数字を文字に戻して表示
print(decode(generated_output))

--- AI店長の生成テキスト ---

店の給盛め番きごでき通」機を使っラま黙だ。食水た玉カウ券トイごちメビ番こでは塩ラーーりやてくれる塩ね。食」めメン上りやかやしゃ待食ゃて待べ硬さいせ替うビまごってないん客：大盛げ置いを使ね店あそ機を番出
店の給水選の給ごューに番サどこで買外ですかよくれ？
客ね硬で。替え外？
がトだ。のごう、上く」が崩店：玉塩へフく見て座せん客へ盛普が崩て大るたう崩ってな。
「普使ってってなあそうませんルフサき麺普ル番味の給そうさませよ通せん「外ラおを願く見な。をよ黙機べまーーメンしか？使バ使願いっ番待塩へ：水機通」が崩あそこ右黙っし何し何ューにね機あすめーべまいせんえ外右まですを外客へいん客味へ見さいせきまいよ


# ==============================================================================
# 🎮 パート5：リアルタイムで会話してみよう (Interactive Chat)
# ==============================================================================

In [8]:
import sys

def chat_with_ai(start_text):
    # 1. ユーザーの入力を数字(トークン)に変換
    try:
        context = torch.tensor([encode(start_text)], dtype=torch.long)
    except KeyError:
        print("エラー: 学習データにない文字が含まれています。")
        return

    # 2. AIに続きを書かせる (50文字くらい)
    print(f"あなた: {start_text}", end="")
    sys.stdout.flush()
    
    # 生成開始
    generated_indices = model.generate(context, max_new_tokens=50)[0].tolist()
    
    # 3. 数字を文字に戻す (入力した部分はカットして、AIの生成部分だけ表示)
    full_text = decode(generated_indices)
    ai_response = full_text[len(start_text):] # 入力部分を削除
    
    print(f"AI店長: {ai_response}")
    print("-" * 30)

# 会話ループ
print("AI店長と会話できます。「終了」と打つと終わります。")
print("コツ: 「客：」から書き始めると、AIが「店：」と答えやすくなります。")
print("-" * 30)

while True:
    user_input = input("入力してください (例: 客：おすすめは？) >> ")
    if user_input == "終了":
        break
    
    # 会話を実行
    chat_with_ai(user_input)

AI店長と会話できます。「終了」と打つと終わります。
コツ: 「客：」から書き始めると、AIが「店：」と答えやすくなります。
------------------------------


入力してください (例: 客：おすすめは？) >>  終了


<br>
<br>
<br>
<hr>

# 🆘 お助けコーナー (Rescue Cell / Answer Key)

もしエラーが出て動かなくなっても大丈夫です！
下のコードは**正解**です。コピーして「パート3」のセルに貼り付けてください。

```python
# 🆘 正解コード (Answer Key)

# 設定
max_iters = 3000  # 学習回数は多いほうが賢くなります

print(f"{max_iters}回の学習を開始します...")

for iter in range(max_iters):
    xb, yb = get_batch('train')

    # 1. 誤差計算
    logits, loss = model(xb, yb)
    
    # 2. 掃除 (答え: optimizer.zero_grad)
    optimizer.zero_grad(set_to_none=True)

    # 3. 逆伝播
    loss.backward()
    
    # 4. 更新 (答え: optimizer.step)
    optimizer.step()

    if iter % 500 == 0:
        print(f"回数: {iter}, 誤差(Loss): {loss.item():.4f}")

print("学習完了！")

<br>
<hr>

# 🎉 午前の部：終了！ (Morning Session Complete)

お疲れ様でした！これで「初めての言語モデル」の構築は完了です。

### ✅ 午前中に学んだこと (Key Takeaways)
1.  **AIは魔法ではない**: 巨大な「確率の計算機」であり、次に来る文字を予測しているだけです。
2.  **学習のサイクル**: `予測` → `間違い(Loss)を計算` → `修正(Optimizer)` を何千回も繰り返すことで賢くなります。
3.  **Loss (損失)**: この数字が下がれば下がるほど、AIはラーメン屋の口調を真似できるようになりました。

---

### 🤔 でも、AIの頭が悪くないですか？ (The Problem)
今のAI（Bigramモデル）は、単語の綴り（スペル）は覚えたようですが、会話の内容はまだ支離滅裂です。

**なぜでしょうか？**

それは、このAIが **「金魚 (Goldfish)」** だからです。
記憶力が「直前の1文字」しかありません。「いらっしゃい」と言うために、「い」だけを見て「ら」を予測しています。前の会話なんて忘れています。

### 🚀 午後の予告：脳の移植手術 (Coming Up Next)
お昼休憩のあと、このAIに **「脳の移植手術」** を行います。

午後の部では、GoogleやOpenAIが採用している技術 **「Transformer (トランスフォーマー)」** を導入します。
AIに「記憶力（文脈を読む力）」を与えると、会話がどう劇的に変わるのか...お楽しみに！

では、お昼休みに入りましょう！🍜

---
# 🍜 頑固ラーメン屋のAIを作ろう (Build a Stubborn Ramen AI)

---
```text
 FFFFF   A   QQQQ      BBBB   OOO  TTTTT
 F      A A  Q  Q      B  B  O   O   T
 FFF   AAAAA Q  Q      BBBB  O   O   T
 F     A   A Q  Q      B  B  O   O   T
 F     A   A  QQ Q     BBBB   OOO    T
```
# 🚀 午後の部：AIの「脳」をアップグレードしよう (Level 2)

午前のモデルは「直前の1文字」しか見ていませんでした（金魚の記憶力）。
これでは「ラーメン」と言いたいのに、「ラ」の次に「ク」が来て「ラクダ」になってしまうかもしれません。

午後は、GoogleやOpenAIが使っている技術**「Transformer (トランスフォーマー)」**を導入します。

### 変わること (Changes)
1. **Self-Attention (注意機構)**: AIが「過去の会話」を振り返るようになります。
2. **Context (文脈)**: 「いらっしゃい」と言われたら「ませ」と返す、といった長い繋がりを理解します。
3. **Complexity (複雑さ)**: コードが長くなりますが、やることは同じ「次に来る文字の予測」です。

---

# ==============================================================================
# ✅ ステップ1 (午後)：設定の更新 (Update Settings)
# ==============================================================================

In [ ]:
# 脳のスペックを上げます
batch_size = 32  # 一度に処理する量
block_size = 32  # 記憶力 (午前は8文字でしたが、午後は32文字まで覚えます！)
max_iters = 3000 # 学習回数
learning_rate = 1e-3
n_embd = 64      # ベクトルの次元数
n_head = 4       # ヘッド数
n_layer = 4      # レイヤー数
dropout = 0.0    # ドロップアウト率

In [ ]:
# ------------------------------------------------------------------------------
# データの読み込み方も、長い記憶力に合わせて更新します
# ------------------------------------------------------------------------------
def get_batch(split):
    # block_size (=32) を使うように変更
    data = torch.tensor(encode(text), dtype=torch.long)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

print(f"設定完了！記憶力が {block_size} 文字にアップグレードされました。")

# ==============================================================================
# ✅ ステップ2 (午後)：GPT トランスフォーマー (新しい脳)
# ==============================================================================
# 警告: ここはコードが長いですが、コピー＆ペーストでOKです！

In [ ]:
class Head(nn.Module):
    """ アテンション・ヘッド：過去の重要な情報を探す機能 """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   
        q = self.query(x) 
        # ここで過去の文字との関連性を計算しています (Search)
        wei = q @ k.transpose(-2,-1) * C**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """ 複数のヘッドで同時に考える機能 """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out

class FeedFoward(nn.Module):
    """ 考えたことを整理する機能 """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ これらをひとまとめにしたブロック """
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
# ==============================================================================
# これが新しい AI モデルです (GPT)
# ==============================================================================
class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # 文字をベクトルに変換
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # 「文字の位置」を覚える機能 (Position Embedding)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # トランスフォーマーのブロックを積み重ねる
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) 
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) 
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device)) 
        x = tok_emb + pos_emb 
        x = self.blocks(x) 
        x = self.ln_f(x) 
        logits = self.lm_head(x) 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # 過去 block_size 分の文字だけを見るように切り取る
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

print("GPTモデル（トランスフォーマー）の定義完了！")

# ==============================================================================
# ✅ ステップ3 (午後)：新しい脳でトレーニング
# ==============================================================================

In [ ]:
# 新しい GPTモデルを初期化
model = GPTLanguageModel()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

print(f"レベルアップした脳で {max_iters} 回の学習を開始します...")
print("ヒント: Lossが 2.0 を切ると、かなり賢くなります。")

for iter in range(max_iters):
    # 午後用のデータを取得 (32文字の記憶)
    xb, yb = get_batch('train')

    # 学習ステップ (午前と同じことを繰り返します)
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if iter % 500 == 0:
        print(f"回数: {iter}, 誤差(Loss): {loss.item():.4f}")

print(f"学習完了！ 最終Loss: {loss.item():.4f}")

# ==============================================================================
# ✅ ステップ4 (午後)：完成したAIと話そう
# ==============================================================================

In [ ]:
import sys

# チャット関数 (午前と同じですが、今の賢い model を使います)
def chat_with_smart_ai(start_text):
    try:
        context = torch.tensor([encode(start_text)], dtype=torch.long)
    except KeyError:
        print("エラー: 学習データにない文字が含まれています。")
        return

    print(f"あなた: {start_text}", end="")
    sys.stdout.flush()
    
    # 生成 (午後は少し長く喋らせてみましょう)
    generated_indices = model.generate(context, max_new_tokens=100)[0].tolist()
    
    full_text = decode(generated_indices)
    ai_response = full_text[len(start_text):]
    
    print(f"AI店長(GPT): {ai_response}")
    print("-" * 30)

print("--- 頑固ラーメンGPT ---")
print("終了するには「終了」と入力してください")

while True:
    user_input = input("入力してください (例: 客：おすすめは？) >> ")
    if user_input == "終了" or user_input == "exit":
        break
    
    chat_with_smart_ai(user_input)

In [ ]:
import sys
def chat_with_smart_ai2(start_text):
    try:
        context = torch.tensor([encode(start_text)], dtype=torch.long)
    except KeyError:
        print("エラー: 学習データにない文字が含まれています。")
        return

    print(f"あなた: {start_text}", end="")
    sys.stdout.flush()
    
    # 生成 (Generate)
    generated_indices = model.generate(context, max_new_tokens=100)[0].tolist()
    full_text = decode(generated_indices)
    
    # 入力した部分を取り除く
    ai_response = full_text[len(start_text):]
    
    # =================================================================
    # ✂️ 追加コード (The Cut-off Logic)
    # AIが勝手に「客：」のセリフを書き始めたら、そこで切り落とします
    # =================================================================
    if "客：" in ai_response:
        ai_response = ai_response.split("客：")[0]
    
    print(f"AI店長(GPT): {ai_response}")
    print("-" * 30)

print("--- 頑固ラーメンGPT ---")
print("終了するには「終了」と入力してください")

while True:
    user_input = input("入力してください (例: 客：おすすめは？) >> ")
    if user_input == "終了" or user_input == "exit":
        break
    
    chat_with_smart_ai2(user_input)

<br>
<br>
<hr>

# 🎓 ワークショップ完了：卒業おめでとうございます！

長い一日、お疲れ様でした！
皆さんは今日、Pythonを使って「何も知らない状態」から「文脈を読むAI」への進化を目撃しました。

### 🏆 今日の成果 (Summary)

| | 午前：Bigram (金魚) | 午後：Transformer (GPT) |
| :--- | :--- | :--- |
| **記憶力** | 直前の1文字だけ | 過去32文字 (文脈) |
| **Loss (誤差)** | 2.5 付近で限界 | **1.5 以下** に激減！ |
| **会話能力** | 宇宙語・支離滅裂 | 「塩ラーメン」を文脈で理解 |
| **正体** | 次の文字のサイコロ | **高度な次の文字予測機** |

### 💡 最後に：AIが「勝手に客のセリフ」を喋った理由
最後のチャット実験で、AIが勝手に「客：水ください」と続きを書き始めませんでしたか？

これが **LLM (大規模言語モデル)** の本質です。
AIは「質問に答えるロボット」ではなく、 **「与えられたテキストの続きをひたすら書く小説家」** なのです。

私たちが普段使っている ChatGPT などは、AIが勝手に続きを書かないように、人間がプログラムで「ストップ！」と命令しているだけなんですね。

---
### 🏠 宿題 (Next Steps)
このノートブックはあなたのものです。家に帰って以下を試してみてください。
1. **データ量**: ラーメン屋のテキストをもっと長く書いてみる。
2. **別の性格**: 関西弁のデータに変えてみる。
3. **脳の拡大**: `n_layer` や `n_head` の数字を大きくしてみる（PCが重くなりますが賢くなります）。

本日はありがとうございました！🍜